In [1]:
import pandas as pd
from pathlib import Path
import seaborn as sns
import numpy as np

In [2]:
def binarize_labels(df):
    df.iloc[:, :3] = df.iloc[:, :3].round()
    return df

In [3]:
dfs = []
# for p in Path('annot_round1').glob('*/batch*.tsv'):
for p in Path('annot_round1').glob('deu_Latn/batch*.tsv'):
    df = pd.read_csv(p, sep='\t')    
    if p.parent.name == 'deu_Latn' and p.name == 'batch0.tsv':
        print('Binarization  for p')
        df = binarize_labels(df)
    df = df.rename(columns = {c: c.split('?')[0] for c in df.columns})
    if any(df[c].isnull().all() for c in df.columns if 'lang' in c):
        continue
    df['path'] = str(p)
    df['lang'] = p.parent.name
    dfs.append(df)
df = pd.concat(dfs, ignore_index=True).sort_values(by='lang')
df.sample(10)

Binarization  for p


,porn,unnatural,lang correct,text_show,id,path,lang
165,NaN,0.0,1.0,Sie sind hier Home >> Die Place Saint-Lambert ...,0db6e302901c4a5642fff986c8e4ec39,annot_round1/deu_Latn/batch0.tsv,deu_Latn
196,NaN,0.0,1.0,Wethmar holt bei der Stadtmeisterschaft im Jug...,c95ec7579cd554fb2e92d6a8137b2c7e,annot_round1/deu_Latn/batch0.tsv,deu_Latn
65,NaN,0.0,2.0,Remi Hotel - Alanya Guzelyali Caddesi Nergis S...,5eb03a22101cd34008bf5b98f834ea22,annot_round1/deu_Latn/batch0.tsv,deu_Latn
41,NaN,0.0,1.0,SPIEGEL ONLINE - 14.07.2014 Der vierte WM-Tite...,ddf64da3499625e0593aa86eff0d3024,annot_round1/deu_Latn/batch0.tsv,deu_Latn
13,NaN,0.0,1.0,Engelbert Broda Engelbert Broda (* 29. August ...,3164209b8d6b734f7c98ceeaa8b6bebe,annot_round1/deu_Latn/batch0.tsv,deu_Latn
20,NaN,NaN,1.0,Beratung und Entsorgungskonzepte Individuelle ...,675c8167f7bc67a7e211f9cabeedc252,annot_round1/deu_Latn/batch0.tsv,deu_Latn
54,NaN,0.0,1.0,"Egal ob Popstar, Filmidol oder Fußballgröße – ...",fbac6890e5f140bb3dde9273d223a9b7,annot_round1/deu_Latn/batch0.tsv,deu_Latn
127,NaN,0.0,1.0,EU bemängelt Informationen über Online-Kredite...,bdb623d6fe2c458603542b01b84703c5,annot_round1/deu_Latn/batch0.tsv,deu_Latn
168,NaN,0.0,1.0,Januar 2013: Tocotronic mit neuem Album “Wie w...,677ca3182e726bf2b2c857519c2cf11b,annot_round1/deu_Latn/batch0.tsv,deu_Latn
162,NaN,0.0,1.0,Um den vollen Funktionsumfang dieser Webseite ...,4e07652c85b9ae3792799a5b96756dcc,annot_round1/deu_Latn/batch0.tsv,deu_Latn


In [4]:
print(len(df), 'examples loaded')
print('Proportions of NaNs:')
df.groupby(['lang','path']).agg(lambda s: s.isnull().mean()).reset_index()


200 examples loaded
Proportions of NaNs:


,lang,path,porn,unnatural,lang correct,text_show,id
0,deu_Latn,annot_round1/deu_Latn/batch0.tsv,0.985,0.025,0.0,0.0,0.0


In [5]:
df = df.dropna(subset=['lang correct'])
print(len(df), 'examples with language annotations')

200 examples with language annotations


In [6]:
df.iloc[:,:3].value_counts(dropna=False).reset_index()

,porn,unnatural,lang correct,count
0,NaN,0.0,1.0,188
1,NaN,1.0,1.0,4
2,1.0,NaN,1.0,2
3,NaN,0.0,0.0,2
4,0.0,NaN,1.0,1
5,NaN,0.0,2.0,1
6,NaN,NaN,0.0,1
7,NaN,NaN,1.0,1


In [7]:
df.iloc[:,:3] = df.iloc[:,:3].fillna(False).replace(1.0, True).replace(0.0, False)
df.iloc[:,:3].value_counts(dropna=False).reset_index()

,porn,unnatural,lang correct,count
0,False,False,True,190
1,False,True,True,4
2,False,False,False,3
3,True,False,True,2
4,False,False,2.0,1


In [8]:
wrong_labels = ~df.iloc[:,:3].isin({True, False}).all(axis=1)
print(wrong_labels.sum(), 'examples with wrong labels:')
df[wrong_labels]

1 examples with wrong labels:


,porn,unnatural,lang correct,text_show,id,path,lang
65,False,False,2.0,Remi Hotel - Alanya Guzelyali Caddesi Nergis S...,5eb03a22101cd34008bf5b98f834ea22,annot_round1/deu_Latn/batch0.tsv,deu_Latn


In [9]:
df.drop(index=df[wrong_labels].index, inplace=True)
print(len(df), 'examples after removing wrong labels')

199 examples after removing wrong labels


In [10]:
def ci(s):
    n = len(s)
    m = s.mean()
    d = np.sqrt(m*(1-m) / n)
    l,r = max(0,m-2*d), min(1.0,m+2*d)
    ci = '---' if m*n < 15 or (1-m)*n < 15 else f'{l:.2f}-{r:.2f}'
    return f'{m:.2f} ({ci})'

In [11]:
pdf = df.drop(columns=['text_show','id','path'])
pdf.groupby(['lang']).agg(lambda s: ci(s)).reset_index()

,lang,porn,unnatural,lang correct
0,deu_Latn,0.01 (---),0.02 (---),0.98 (---)


# By group

In [20]:
df

,porn,unnatural,lang correct,text_show,id,path,lang
1299,False,False,True,مصر: جلسة طارئة لنقابة المحاميين لطرح الثقة او...,bc745a22a8161536d6cb1a9bd9afe10b,annot_round1/ara_Arab/batch0.tsv,ara_Arab
1270,False,False,True,لاهاي –رويترز: يتوقع ان يحاول سيف الاسلام القذ...,c5475014b89b0533744d1370554dbf31,annot_round1/ara_Arab/batch0.tsv,ara_Arab
1269,False,False,True,جالاكسي نوت 10.1 يظهر اليوم في الاسواق الامريك...,d85701b14a066a3d23c5dfbf8c415490,annot_round1/ara_Arab/batch0.tsv,ara_Arab
1268,False,False,True,المقالات - التفاصيل - تاريخ النشر - كتب بواسطة...,339c8c24161fa6b5eed6f0559aed747b,annot_round1/ara_Arab/batch0.tsv,ara_Arab
1267,False,False,True,خطبة المسجد النبوي - 16 ربيع الثاني 1433 - الج...,9b289bb8624e1d209aa55b638a72644d,annot_round1/ara_Arab/batch0.tsv,ara_Arab
...,...,...,...,...,...,...,...
471,False,False,True,Mantarli Hindi – Yemek Tarifi Mutfak: Bilinmiy...,5a3ab3214cf6a92123e6ef3807b710ba,annot_round1/tur_Latn/batch0.tsv,tur_Latn
472,False,False,False,Makale özeti ve diğer detaylar. Sekreterlik me...,893fefe2c7f35361e97871aa110bb4a6,annot_round1/tur_Latn/batch0.tsv,tur_Latn
473,False,False,True,Asgari ücret ne kadar artacak? 2013 Yılı Progr...,e0be0d5abe6bf89d7ecfe99cb60bad14,annot_round1/tur_Latn/batch0.tsv,tur_Latn
463,False,False,True,MUĞLA'nın Seydikemer İlçesi'ne gelen Milliyetç...,23131da24b26cff33d264567a6939085,annot_round1/tur_Latn/batch0.tsv,tur_Latn
